In [1]:
import os
!pwd

/Users/tejasgadi/Local_Disk_D/Machine_Learning_Projects/finetuning_distilbert_text_classification/research


In [2]:
# Goto Root directory of the project
os.chdir("../")
!pwd

/Users/tejasgadi/Local_Disk_D/Machine_Learning_Projects/finetuning_distilbert_text_classification


In [3]:
# Model trauiner Config Schema/Data Class
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [4]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories
from box import ConfigBox

## Config Manager

In [5]:
# Create Configuration Manager: This has basic information required before staring any module/components

class ConfigurationManager:
    def __init__(self, config_path = CONFIG_FILE_PATH, params_path = PARAMS_FILE_PATH):
        self.config:ConfigBox = read_yaml(config_path)
        self.params:ConfigBox = read_yaml(params_path).TrainingArguments

        # Create Artifacts Root dir
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self)->ModelTrainerConfig:
        config = self.config.model_trainer

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=self.params.num_train_epochs,
            warmup_steps=self.params.warmup_steps,
            per_device_train_batch_size=self.params.per_device_train_batch_size,
            weight_decay=self.params.weight_decay,
            logging_steps=self.params.logging_steps,
            eval_strategy=self.params.eval_strategy,
            eval_steps=self.params.eval_steps,
            save_steps=self.params.save_steps,
            gradient_accumulation_steps=self.params.gradient_accumulation_steps
        )

        return model_trainer_config


In [6]:
## Model Trainer libraries import
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk

/Users/tejasgadi/Local_Disk_D/Machine_Learning_Projects/finetuning_distilbert_text_classification/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-28 16:57:43,916 : INFO: config: PyTorch version 2.6.0 available.]


## Model Trainer component

In [ ]:
class ModelTrainer:
    def __init__(self, data_transformation_config: ModelTrainerConfig):
        self.config = data_transformation_config

    # Training logic
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)

        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)

        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        # loading the data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            eval_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16,
            no_cuda=True
        )

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train(v)

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

In [10]:
# Pipeline initiate

config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()

model_trainer = ModelTrainer(model_trainer_config)

model_trainer.train()

[2025-04-28 16:58:28,147 : INFO: common: yaml file: config/config.yaml is loaded sucessfully]
[2025-04-28 16:58:28,151 : INFO: common: yaml file: params.yaml is loaded sucessfully]
[2025-04-28 16:58:28,152 : INFO: common: Created directory at: artifacts]
[2025-04-28 16:58:28,152 : INFO: common: Created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/tejasgadi/Local_Disk_D/Machine_Learning_Projects/finetuning_distilbert_text_classification/venv/lib/python3.12/site-packages/transformers/training_args.py:1595: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
/var/folders/hq/tbxvw3vn6wb7ky65zh2vrfm40000gn/T/ipykernel_33291/333908283.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args=trainer_args,


Step,Training Loss,Validation Loss


/Users/tejasgadi/Local_Disk_D/Machine_Learning_Projects/finetuning_distilbert_text_classification/venv/lib/python3.12/site-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


RuntimeError: [enforce fail at inline_container.cc:626] . unexpected pos 4499353088 vs 4499352988